# [Day-10](https://adventofcode.com/2019/day/10) - Finding most visible asteroid for a base.

In [1]:
import math
import numpy as np
import pandas as pd

asteroids_map = pd.read_csv("./input/Day-10.txt", header=None)[0].str.extractall('(.)')[0].unstack().values
asteroids_set = set((x,y) for y,x in zip(*(asteroids_map == '#').nonzero())) # Flips coordinates to match instructions.

def in_line_of_sight(a,b):
    """Calculates the number of integer locations in line of sight between a & b."""
    if a == b: return []
    diff = np.array([b[0] - a[0], b[1] - a[1]])
    if not diff[0]:   sight = [(a[0], y) for y in range(a[1], b[1], (b[1] - a[1]) // abs(b[1] - a[1]))][1:]
    elif not diff[1]: sight = [(x, a[1]) for x in range(a[0], b[0], (b[0] - a[0]) // abs(b[0] - a[0]))][1:]
    else:
        min_integer_jump = min(abs(diff)) / math.gcd(*diff)
        jumps = ((diff / min(abs(diff))) * min_integer_jump).astype(int)
        sight = list(zip(range(a[0],b[0],jumps[0]), range(a[1],b[1],jumps[1])))[1:]
    return sight

def calc_azimuth(a,b):
    """Calculate the clockwise azimuth angle (in radians) & distance between from a to b."""
    diff = np.array([a[0] - b[0], a[1] - b[1]])
    diff_unit = diff / np.linalg.norm(diff)
    radians = np.arccos(np.clip(np.dot([0,1], diff_unit), -1.0, 1.0))
    radians = radians if diff[0] <= 0 else np.pi * 2 - radians # Fixes radians to make some radians greater than np.pi
    return (np.around(radians, 8), np.linalg.norm(diff))

def blow_up_astroids(base, num_astroids_to_blow_up=200):
    # Computing angles from base to all asteroids.
    angles_from_base = sorted([(*calc_azimuth(base, asteroid), asteroid) 
                               for asteroid in asteroids_set if asteroid != base])
    # Stack astroids from each angle to the asteroids in this angle.
    angles_to_asteroids_dict = dict()
    for angle, _, asteroid in angles_from_base:
        if angle not in angles_to_asteroids_dict:
            angles_to_asteroids_dict[angle] = []
        angles_to_asteroids_dict[angle].append(asteroid)
    # Blow up the astroids rotating the laser clockwise after each explosion to next astroid.
    angles_to_meteros_keys = sorted(angles_to_asteroids_dict.keys())
    i, current_angle_pointer = 1, 0
    while i <= num_astroids_to_blow_up:
        angle = angles_to_meteros_keys[current_angle_pointer]
        if angles_to_asteroids_dict[angle]:
            exploded_asteroid = angles_to_asteroids_dict[angle].pop(0)
            if i == num_astroids_to_blow_up:
                return exploded_asteroid
            i += 1
        current_angle_pointer = current_angle_pointer + 1 % len(angles_to_meteros_keys)
    
# Find the asteroid viewing the most other astroids.
def solve10a_in_line():
    # Faster, but relies on coordinates being integers.
    return max((sum(not any(asteroids_set.intersection(in_line_of_sight(chosen_base, asteroid))) 
                    for asteroid in asteroids_set if asteroid != chosen_base), 
                chosen_base) for chosen_base in asteroids_set)
def solve10a_azimuth():
    # Slower, but works for float coordinates, and reuses code from 10b.
    return max((len(set(calc_azimuth(chosen_base, asteroid)[0] 
                        for asteroid in asteroids_set if asteroid != chosen_base)),
                chosen_base) for chosen_base in asteroids_set)

visible_astroids, chosen_base = solve10a_in_line()
print(f"10a (in line). The chosen base is in {chosen_base}, viewing [{visible_astroids}] other astroids.")
visible_astroids, chosen_base = solve10a_azimuth()
print(f"10a (azimuth). The chosen base is in {chosen_base}, viewing [{visible_astroids}] other astroids.")
print(f"10b. The 200th astroid blown up from {chosen_base} was {blow_up_astroids(chosen_base)}.") 

10a (in line). The chosen base is in (20, 19), viewing [284] other astroids.
10a (azimuth). The chosen base is in (20, 19), viewing [284] other astroids.
10b. The 200th astroid blown up from (20, 19) was (4, 4).


In [2]:
%timeit solve10a_in_line()

1.06 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
%timeit solve10a_azimuth()

5.58 s ± 124 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit blow_up_astroids(chosen_base)

14.9 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
